In [ ]:
# pip install -U flask-cors tensorflow nltk
# nltk.download('punkt')
# nltk.download('wordnet')

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import random

lemmatizer = WordNetLemmatizer()

In [2]:
from tensorflow.keras.models import load_model
import json

model = load_model('chatbot_model_2.h5')
data_file = open('intents.json').read()
intents = json.loads(data_file)
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [3]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence words: ", sentence_words)
    return sentence_words

In [4]:
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))

In [5]:
def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    print("Res:",res)
    ERROR_THRESHOLD = 0.25
    
    result = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    print("result: ", result)
    result.sort(key=lambda x: x[1], reverse= True)
    result_list = []
    for r in result:
        result_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print("result list: ",result_list)
        
    return result_list

In [6]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
    return result

In [7]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    try:
        res = getResponse(ints, intents)
    except:
        res = "I am really sorry, I don't have the knowlegde to answer this."
    return res
    

In [8]:
chatbot_response("What can you do ?")

sentence words:  ['what', 'can', 'you', 'do', '?']
1/1 [==============================] - 0s 74ms/step
Res: [5.5499876e-11 2.2562983e-07 2.4492470e-09 1.7135552e-13 7.7653724e-14
 4.0234647e-09 2.0424840e-09 5.1089575e-08 3.5161101e-08 1.7334700e-09
 1.2122756e-09 5.8025451e-10 2.7752083e-08 3.3707386e-07 7.8891240e-09
 8.2654923e-09 2.6429905e-08 1.1761605e-11 9.9999893e-01 1.2393508e-10
 1.6562544e-09 3.1134869e-07 2.0925439e-09 1.6510256e-12]
result:  [[18, 0.9999989]]
result list:  [{'intent': 'options', 'probability': '0.9999989'}]


'I am a general-purpose chatbot. Here are some things I can assist you with:\n\n1. Answering machine learning questions\n2. Providing the current date and time\n3. Setting timers\n\nFeel free to ask any questions or make specific requests!'

In [9]:
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/", methods = ['GET'])
def hello():
    return jsonify({"Server": "Sucessfully"})

def decrypt(msg):
    string = msg
    new_string = string.replace("+", " ")
    return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    dec_msg = decrypt(sentence)
    
    response = chatbot_response(dec_msg)
    
    json_obj = jsonify({"top":{"res": response}})
    
    return json_obj

In [10]:
# import logging
# from flask import Flask, jsonify
# from flask_ngrok import run_with_ngrok
# from flask_cors import CORS

# app = Flask(__name__)
# run_with_ngrok(app)
# CORS(app)

# @app.route("/", methods = ['GET'])
# def index():
#     return jsonify({"Server": "Sucessfully"})

# def decrypt(msg):
#     string = msg
#     new_string = string.replace("+", " ")
#     return new_string

# @app.route("/query/<sentence>")
# def query_chatbot(sentence):
#     dec_msg = decrypt(sentence)
    
#     response = chatbot_response(dec_msg)
    
#     json_obj = jsonify({"top":{"res": response}})
    
#     return json_obj

# if __name__ == '__main__':
#     app.run()

In [11]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


sentence words:  ['hello']
1/1 [==============================] - 0s 16ms/step


127.0.0.1 - - [06/Oct/2023 20:56:59] "GET /query/Hello HTTP/1.1" 200 -


Res: [1.1877989e-06 4.6002506e-06 4.8907219e-07 9.8358860e-06 1.9054772e-06
 4.4123016e-07 1.6034752e-05 5.5297319e-06 1.6900438e-08 5.4720990e-06
 3.0414151e-06 9.9992299e-01 7.2451609e-09 1.2483484e-05 2.4977814e-07
 2.2732258e-09 7.1833270e-08 2.0893862e-07 3.9749685e-07 1.3073817e-06
 1.0763643e-05 3.7263270e-08 1.2031605e-06 1.6656237e-06]
result:  [[11, 0.999923]]
result list:  [{'intent': 'greeting', 'probability': '0.999923'}]
sentence words:  ['what', 'can', 'you', 'do', '?']
1/1 [==============================] - 0s 15ms/step


127.0.0.1 - - [06/Oct/2023 20:57:04] "GET /query/What%20can%20you%20do%20%3F HTTP/1.1" 200 -


Res: [5.5499876e-11 2.2562983e-07 2.4492470e-09 1.7135552e-13 7.7653724e-14
 4.0234647e-09 2.0424840e-09 5.1089575e-08 3.5161101e-08 1.7334700e-09
 1.2122756e-09 5.8025451e-10 2.7752083e-08 3.3707386e-07 7.8891240e-09
 8.2654923e-09 2.6429905e-08 1.1761605e-11 9.9999893e-01 1.2393508e-10
 1.6562544e-09 3.1134869e-07 2.0925439e-09 1.6510256e-12]
result:  [[18, 0.9999989]]
result list:  [{'intent': 'options', 'probability': '0.9999989'}]
